01ML ===>>> From Item_Id Recommend 5 Games  

In [1]:
import pandas as pd
from fastapi import FastAPI

In [3]:
df_recommendation= pd.read_csv(r"C:\Users\m_evi\OneDrive\Desktop\MLInt\Datasets\CleanDatasets\steam_games.csv")


In [4]:
df_recommendation.head()

,genres,price,early_access,item_id,release_year,publisher,app_name,item_name,developer
0,Action,4.99,0.0,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,Casual,4.99,0.0,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
2,Indie,4.99,0.0,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
3,Simulation,4.99,0.0,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
4,Strategy,4.99,0.0,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [9]:


def game_recommendation(item_id: int):

  
    # Filter games that match the given item ID
    filtered_game = df_recommendation[df_recommendation['item_id'] == item_id]

    # Get the genres of the given game
    game_genres = set(filtered_game['genres'].str.split(',').explode())

    # Filter games that share at least 1 genre with the given game
    recommended_games = df_recommendation[df_recommendation['genres'].apply(lambda x: len(set(x.split(',')).intersection(game_genres)) >= 1)]

    # Calculate cosine similarity between genre vectors of games
    recommended_games.loc[:, 'genres_vector'] = recommended_games['genres'].apply(lambda x: np.array([1 if genre in x else 0 for genre in game_genres]))
    filtered_game.loc[:, 'genres_vector'] = filtered_game['genres'].apply(lambda x: np.array([1 if genre in x else 0 for genre in game_genres]))
    recommended_games.loc[:, 'similarity'] = recommended_games.apply(lambda row: cosine_similarity([row['genres_vector']], [filtered_game['genres_vector'].iloc[0]])[0][0], axis=1)

    # Sort games by similarity and recommendation in descending order
    recommended_games = recommended_games.sort_values(['similarity'], ascending=[False])

    # Select the top 5 games with highest similarity and recommendation
    top_recommended_games = recommended_games.head(5)

    # Create a dictionary of recommended game names along with their developers
    recommended_games_dict = {}
    recommended_games_dict['Because you liked ' + filtered_game['item_name'].iloc[0] + ', you might also enjoy...'] = top_recommended_games[['item_name']].to_dict(orient='records')

    return recommended_games_dict

In [10]:
game_recommendation(70)

C:\Users\m_evi\AppData\Local\Temp\ipykernel_4452\2618191416.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_games.loc[:, 'genres_vector'] = recommended_games['genres'].apply(lambda x: np.array([1 if genre in x else 0 for genre in game_genres]))
C:\Users\m_evi\AppData\Local\Temp\ipykernel_4452\2618191416.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_game.loc[:, 'genres_vector'] = filtered_game['genres'].apply(lambda x: np.array([1 if genre in x else 0 for genre in game_genres

{'Because you liked Half-Life, you might also enjoy...': [{'item_name': 'Lost Summoner Kitty'},
  {'item_name': 'Pale Blue Gullwings &amp; Pale Blue Visor'},
  {'item_name': 'Polarity - Soundtrack'},
  {'item_name': 'Hikikomori No Chuunibyou'},
  {'item_name': 'Gazzel Quest, The Five Magic Stones'}]}

In [ ]:
game_recommendation(1250,df_recommendation)

C:\Users\m_evi\AppData\Local\Temp\ipykernel_13904\3183896075.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_games.loc[:, 'genres_vector'] = recommended_games['genres'].apply(lambda x: np.array([1 if genre in x else 0 for genre in game_genres]))
C:\Users\m_evi\AppData\Local\Temp\ipykernel_13904\3183896075.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_game.loc[:, 'genres_vector'] = filtered_game['genres'].apply(lambda x: np.array([1 if genre in x else 0 for genre in game_genr

{'Because you liked Killing Floor, you might also enjoy...': [{'item_name': 'Lost Summoner Kitty'},
  {'item_name': 'Pale Blue Gullwings &amp; Pale Blue Visor'},
  {'item_name': 'Polarity - Soundtrack'},
  {'item_name': 'Hikikomori No Chuunibyou'},
  {'item_name': 'Gazzel Quest, The Five Magic Stones'}]}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the DataFrame df_games from your CSV file
df_games = pd.read_csv('./CleanDatasets/steam_games.csv')

def game_recommendation2(product_id: int):
    try:
        # Get the reference game
        target_game = df_games[df_games['item_id'] == product_id]

        if target_game.empty:
            return {"message": "Reference game not found."}
        
        # Create a TF-IDF vectorizer
        tfidf_vectorizer = TfidfVectorizer()

        # Set the batch size for reading games
        chunk_size = 100

        # Initialize similarity scores
        similarity_scores = None

        # Process games in batches using chunks
        for chunk in pd.read_csv('./CleanDatasets/steam_games.csv', chunksize=chunk_size):

            # Apply the TF-IDF vectorizer to the current batch of games and the reference game
            tfidf_matrix = tfidf_vectorizer.fit_transform(chunk['app_name'])

            # Calculate similarity between the reference game and games in the current batch
            if similarity_scores is None:
                similarity_scores = cosine_similarity(tfidf_matrix, tfidf_matrix[0])
            else:
                similarity_scores = cosine_similarity(tfidf_matrix, similarity_scores)

        # Get indices of the most similar games
        similar_games_indices = similarity_scores[0].argsort()[::-1]

        # Recommend the most similar games (you can adjust the number of recommendations)
        num_recommendations = 5
        recommended_games = df_games.loc[similar_games_indices[1:num_recommendations + 1]]

        # Return the list of recommended games
        return recommended_games[['app_name', 'genres']].to_dict(orient='records')

    except Exception as e:
        return {"message": "Sorry, there was an error processing the recommendation."}

# Example usage:
# recommended_games2 = game_recommendation(product_id=12345)
# print(recommended_games)

In [ ]:
recommended_games2 = game_recommendation2(70)
print(recommended_games2)

{'message': 'Sorry, there was an error processing the recommendation.'}


02ML ===>>> From **User_id** recommend 5 similar games with User_id 

In [ ]:
#app2 = FastAPI()
from sklearn.feature_extraction.text import TfidfVectorizer
# Load the DataFrame df_games from your CSV file
df_games = pd.read_csv('./CleanDatasets/steam_games.csv')

#@app2.get("/game_recommendation/{product_id}")
async def game_recommendation(product_id: int):
    try:
        # Get the reference game
        target_game = df_games[df_games['item_id'] == product_id]

        if target_game.empty:
            return {"message": "Reference game not found."}
        
        # Create a TF-IDF vectorizer
        tfidf_vectorizer = TfidfVectorizer()

        # Set the batch size for reading games
        chunk_size = 100

        # Process games in batches using chunks
        for chunk in pd.read_csv('./CleanDatasets/steam_games.csv', chunksize=chunk_size):

            # Apply the TF-IDF vectorizer to the current batch of games and the reference game
            tfidf_matrix = tfidf_vectorizer.fit_transform([target_game])

            # Calculate similarity between the reference game and games in the current batch
            if similarity_scores is None:
                similarity_scores = cosine_similarity(tfidf_matrix)
            else:
                similarity_scores = cosine_similarity(tfidf_matrix, X=similarity_scores)

        if similarity_scores is not None:
            # Get indices of the most similar games
            similar_games_indices = similarity_scores[0].argsort()[::-1]
            # Recommend the most similar games (you can adjust the number of recommendations)
            num_recommendations = 5
            recommended_games = df_games.loc[similar_games_indices[1:num_recommendations + 1]]

            # Return the list of recommended games
            return recommended_games[['app_name', 'genres']].to_dict(orient='records')
        
        return {"message": "Sorry, No similar games found"}

    except Exception as e:
        return {"message": f"Sorry, there is an Error, try again: {str(e)}"} 
    
    

In [ ]:
df_games['item_id'] = df_games['item_id'].astype(str).str.lower()
# Call the function with a specific product ID
product_id = "70"  # replace with your actual product ID
result = await game_recommendation(product_id)

# Print the result
print(result)

{'message': "Sorry, there is an Error, try again: 'DataFrame' object has no attribute 'lower'"}


In [ ]:
game_recommendation(761140)

<coroutine object game_recommendation at 0x000002E1629509C0>

Recommendation Model    